In [ ]:
# %run, %load, 
%load_ext autoreload
%autoreload 2

In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
# from tqdm import tqdm
import time

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True


In [2]:
device = torch.device("cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.1
num_epochs = 4
batch_size = 64

# Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10


In [3]:
# Load data
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [4]:
for images, labels in train_loader:
    print("image batch dimensions:", images.shape) # batch_size, 1, row, column
    print("image label dim:", labels.shape) 
    break

image batch dimensions: torch.Size([64, 1, 28, 28])
image label dim: torch.Size([64])


In [5]:
# Model

class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(MultiLayerPerceptron, self).__init__()

        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1) # Xavier/Glorot init
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)

    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = F.relu(self.linear_2(out))
        logits = self.linear_out(out)
        probs = F.log_softmax(logits, dim=1)
        return logits, probs

torch.manual_seed(random_seed)
model = MultiLayerPerceptron(num_features=num_features, num_classes=num_classes)

model = model.to(device) # put on GPU

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [6]:
def compute_acc(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device) # put on GPU
            targets = targets.to(device)
            logits, probs = net(features)
            _, predicted_labels = torch.max(probs, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float() / num_examples * 100

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)

        # forward and backprop
        logits, probs = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()

        cost.backward()

        # update params
        optimizer.step()

        if not batch_idx % 50:
            print(f'Epoch: {epoch+1:2}/{num_epochs:2} \tBatch: {batch_idx:3}/{len(train_loader):3} \tCost: {cost:.2}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: {compute_acc(model, train_loader):.3}')

    print(f'Time: {((time.time() - start_time) / 60):.2} min \n')


Epoch:  1/ 4 	Batch:   0/938 	Cost: 2.3
Epoch:  1/ 4 	Batch:  50/938 	Cost: 1.7
Epoch:  1/ 4 	Batch: 100/938 	Cost: 0.72
Epoch:  1/ 4 	Batch: 150/938 	Cost: 0.5
Epoch:  1/ 4 	Batch: 200/938 	Cost: 0.39
Epoch:  1/ 4 	Batch: 250/938 	Cost: 0.48
Epoch:  1/ 4 	Batch: 300/938 	Cost: 0.42
Epoch:  1/ 4 	Batch: 350/938 	Cost: 0.14
Epoch:  1/ 4 	Batch: 400/938 	Cost: 0.28
Epoch:  1/ 4 	Batch: 450/938 	Cost: 0.3
Epoch:  1/ 4 	Batch: 500/938 	Cost: 0.38
Epoch:  1/ 4 	Batch: 550/938 	Cost: 0.26
Epoch:  1/ 4 	Batch: 600/938 	Cost: 0.21
Epoch:  1/ 4 	Batch: 650/938 	Cost: 0.37
Epoch:  1/ 4 	Batch: 700/938 	Cost: 0.18
Epoch:  1/ 4 	Batch: 750/938 	Cost: 0.3
Epoch:  1/ 4 	Batch: 800/938 	Cost: 0.12
Epoch:  1/ 4 	Batch: 850/938 	Cost: 0.11
Epoch:  1/ 4 	Batch: 900/938 	Cost: 0.31
training accuracy: 92.9
Time: 0.29 min 

Epoch:  2/ 4 	Batch:   0/938 	Cost: 0.21
Epoch:  2/ 4 	Batch:  50/938 	Cost: 0.2
Epoch:  2/ 4 	Batch: 100/938 	Cost: 0.18
Epoch:  2/ 4 	Batch: 150/938 	Cost: 0.32
Epoch:  2/ 4 	Batch: 2

In [7]:
print('Test accuracy: %.2f%%' % (compute_acc(model, test_loader)))


Test accuracy: 97.61%
